In [1]:
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import json

In [2]:
class Window(QMainWindow):
    def setupMenuBar(self):
        def setupFileMenu(self, mainMenu):
            fileMenu = mainMenu.addMenu("Image")
            
            loadAction = QAction("Load", self)
            fileMenu.addAction(loadAction)
            loadAction.triggered.connect(self.loadFile)

            saveAction = QAction("Save", self)
            saveAction.setShortcut("Ctrl+S")
            fileMenu.addAction(saveAction)
            saveAction.triggered.connect(self.save)

            
            return fileMenu
        
        def setupColorMenu(self, mainMenu):
            def setupColor(name, action):
                colorAction = QAction(name, self)
                colorMenu.addAction(colorAction)
                colorAction.triggered.connect(action)
            
            def blackAction():
                self.brushColor = Qt.black
            
            def whiteAction():
                self.brushColor = Qt.white
            colorMenu = mainMenu.addMenu('Color')
            
            setupColor('Black', blackAction)
            setupColor('White', whiteAction)
            
        def setupFigureMenu(self, mainMenu):
            def setupFigure(name, action):
                figureAction = QAction(name, self)
                figureMenu.addAction(figureAction)
                figureAction.triggered.connect(action)
                
            def penAction():
                self.Fig = 'pen'
            
            def rectangleAction():
                self.Fig = 'rectangle'
            
            figureMenu = mainMenu.addMenu('Figure')
            
            setupFigure('Pen', penAction)
            setupFigure('rectangle', rectangleAction)
            
        mainMenu = self.menuBar()
        setupFileMenu(self, mainMenu)
        setupColorMenu(self, mainMenu)
        setupFigureMenu(self, mainMenu)
        
        return mainMenu
        
    def setupBackground(self):
        self.loadFile()
        
        
    def __init__(self):
        super().__init__()
        
        title = 'Draw Laba 1'
        
        self.setWindowTitle(title)

        self.Fig = 'pen'

        self.isDrawing = False
        self.brushSize = 1
        self.brushColor = Qt.black
        self.lastPoint = QPoint()
        
        self.file = None

        self.label = QLabel(self)

        
        mainMenu = self.setupMenuBar()
        self.setupBackground()

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.isDrawing = True
            self.lastPoint = event.pos()
            self.firstPoint = event.pos()
            print([self.firstPoint, self.lastPoint])
        if self.Fig == 'pen':
            self.file.write(json.dumps(
                {
                    'point':
                    
                        {
                            'x': self.lastPoint.x(),
                            'y': self.lastPoint.y()
                        }
                    
                }
            ))

    def mouseMoveEvent(self, event):
        if (event.buttons() & Qt.LeftButton) & self.isDrawing:
            painter = QPainter(self.image)
            painter.setPen(QPen(self.brushColor, self.brushSize, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            if self.Fig == 'pen':
                painter.drawLine(self.lastPoint, event.pos())
                
                self.file.write(json.dumps(
                {
                    'point':
                    
                        {
                            'x': self.lastPoint.x(),
                            'y': self.lastPoint.y()
                        }
                    
                }
            ))

                

            self.lastPoint = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.isDrawing = False
            
            if self.Fig == 'rectangle':
                painter = QPainter(self.image)
                painter.setPen(QPen(self.brushColor, self.brushSize, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
                painter.drawRect(self.firstPoint.x(), self.firstPoint.y(),
                                 (self.lastPoint.x() - self.firstPoint.x()),
                                 (self.lastPoint.y() - self.firstPoint.y()))
                self.file.write(json.dumps(
                        {'rectangle':
                        
                            {
                                'x1': self.firstPoint.x(),
                                'y1': self.firstPoint.y(),
                                'x2': self.lastPoint.x(),
                                'y2': self.lastPoint.y()
                            }
                        
                        }
                    
                ))
                self.update()
    

    def paintEvent(self, event):
        canvasPainter = QPainter(self)
        canvasPainter.drawImage(self.rect(), self.image, self.image.rect())
        
    def closeEvent(self, event):
        self.file.close()

    def loadFile(self):
        filename = QFileDialog.getOpenFileName(self, 'Open File')
        self.image = QImage(filename[0])
        self.setFixedSize(self.image.width(), self.image.height())
        
        localFileName = filename[0].split('/')[-1].split('.')[0]
        if self.file is not None:
            self.file.close()
            self.file = open(localFileName + '.txt', 'w')
                
        else:
            self.file = open(localFileName + '.txt', 'w')

    def save(self):
        filePath, _ = QFileDialog.getSaveFileName(self, "Save Image")

        if filePath == "":
            return
        self.image.save(filePath)
        self.file.close()

In [ ]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    app.exec()